# Задание 2. Вариант 20.

Создать вычислительный граф функции. Вычислить значение функции для некоторых значений
переменных.

$$ 5x^2 * 8y^2 + 7x^2 * 3y $$

## Оператор

In [84]:
class Operation():
    """
    An Operation is a node in a "Graph". TensorFlow will also use this concept of a Graph.
    
    This Operation class will be inherited by other classes that actually compute the specific
    operation, such as adding or matrix multiplication.
    """
    
    def __init__(self, input_nodes = []):
        """
        Intialize an Operation
        """
        self.input_nodes = input_nodes # The list of input nodes
        self.output_nodes = [] # List of nodes consuming this node's output
        
        # For every node in the input, we append this operation (self) to the list of
        # the consumers of the input nodes
        for node in input_nodes:
            node.output_nodes.append(self)
        
        # There will be a global default graph (TensorFlow works this way)
        # We will then append this particular operation
        # Append this operation to the list of operations in the currently active default graph
        _default_graph.operations.append(self)
  
    def compute(self):
        """ 
        This is a placeholder function. It will be overwritten by the actual specific operation
        that inherits from this class.
        
        """
        
        pass

### Сложение

In [85]:
class add(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var + y_var

### Перемножение

In [86]:
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var

### Возведение в степень

In [87]:
class degree(Operation):
     
    def __init__(self, a, d):
        super().__init__([a, d])
    
    def compute(self, a_var, d_var):
        self.inputs = [a_var, d_var]
        return a_var ** d_var

## Placeholders

In [88]:
class Placeholder():
    """
    A placeholder is a node that needs to be provided a value for computing the output in the Graph.
    """
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

## Переменные

In [89]:
class Variable():
    """
    This variable is a changeable parameter of the Graph.
    """
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)

## Граф

In [90]:
class Graph():
    
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        """
        Sets this Graph instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self

## Создание графа

$$ z = Ax^d * By^d + Cx^d * Dy $$

A = 5; B = 8; C = 7; D = 3; d = 2:

$$ z = 5x^2 * 8y^2 + 7x^2 * 3y $$

In [91]:
g = Graph()

In [92]:
g.set_as_default()

In [93]:
A = Variable(5) 
B = Variable(8)
C = Variable(7)
D = Variable(3)
d = Variable(2)

In [94]:
x = Placeholder()
y = Placeholder()

In [95]:
a = degree(x,d) #x^2
b = degree(y,d) #y^2

In [96]:
k = multiply(A,a) # 5x^2
l = multiply(B,b) # 8y^2
m = multiply(C,a) # 7x^2
n = multiply(D,y) # 3y

In [97]:
kl = multiply(k,l) # 5x^2 * 8y^2 
mn = multiply(m,n) # 7x^2 * 3y

In [98]:
z = add(kl,mn) #5x^2 * 8y^2 + 7x^2 * 3y

## Session

In [99]:
import numpy as np

### Traversing Operation Nodes

In [100]:
def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [101]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output




In [102]:
sess = Session()

In [103]:
# тестовая функция: считает выражение от заданных x и y
def test(x,y):
    return 40 * x**2 * y**2 + 21 * x**2 * y

In [104]:
test_sets = [(1,2), (2,5), (33, 18)]

In [106]:
# сверяем результаты графа с результатами тестовой функции
for i, test_set in enumerate(test_sets): 
    print('{}) x={}\ty={}'.format(i+1,test_set[0],test_set[1]))
    result = sess.run(operation=z,feed_dict={x:test_set[0],y:test_set[1]})
    eval = result == test(test_set[0],test_set[1])
    print('{}: {}'.format(result, eval))

1) x=1	y=2
202: True
2) x=2	y=5
4420: True
3) x=33	y=18
14525082: True
